## Import libraries

In [7]:
import pandas as pd
import numpy as np
pd.__version__

'1.5.2'

## Function definition

In [51]:
def list_to_numeric(x):
    if(len(x) == 1):
        return pd.to_numeric(x, errors='coerce')
    if(len(x) == 2):
        x = pd.to_numeric(x, errors='coerce')
        #big difference in numbers:
        x.sort()
        if(x[0] < x[1]/1000):
            x[0] = x[0]*1000
        if(x[0] == 0):
            x = x[1]*1000
        x = np.mean(x)
        return x
    else:
        return np.nan

In [74]:
def extract_salary(df):
    s = df['description'].str.extractall(r'([\$][ 0123456789,BMbmilkK-]*)')
    s = s.reset_index(names=['id', 'match'])
    s = s.rename(columns={0:'salary_raw'})
    s['salary_str'] = s['salary_raw'].map(lambda x: x.lower().lstrip('$ ').rstrip(' /-.'))
    s['salary_str'] = s['salary_str'].map(lambda x: x.replace(',','').replace('.00', '').replace('-', ' ').replace('k', '000'))
    s['salary_str'] = s['salary_str'].map(lambda x: x.split())
    s['salary_numeric'] = s['salary_str'].apply(list_to_numeric)
    return s

## Load data

In [75]:
df = pd.read_hdf('../data/linkedin_jobs.h5')
df = df.reset_index(drop=True)
# df['description'] = df['description'].str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414 entries, 0 to 1413
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1414 non-null   object
 1   company      1414 non-null   object
 2   link         1414 non-null   object
 3   location     1414 non-null   object
 4   description  1414 non-null   object
 5   date         1414 non-null   object
 6   experience   1414 non-null   object
dtypes: object(7)
memory usage: 77.5+ KB


In [76]:
df.head(5)

,title,company,link,location,description,date,experience
0,Data Analyst,Field Museum,https://www.linkedin.com/jobs/view/data-analys...,"Chicago, IL",The Field Museum is a not-for-profit organizat...,2023-02-26,ENTRY_LEVEL
1,Data Analyst,Ann & Robert H. Lurie Children's Hospital of C...,https://www.linkedin.com/jobs/view/data-analys...,"Chicago, IL",Ann & Robert H. Lurie Children’s Hospital of C...,2023-06-16,ENTRY_LEVEL
2,Excellent Opportunity - Data Analyst in Chicag...,PSRTEK,https://www.linkedin.com/jobs/view/excellent-o...,"Chicago, IL","Job Title: Data Analyst\n\nLocation: Chicago, ...",2023-07-05,ENTRY_LEVEL
3,Data Analyst,LaSalle Network,https://www.linkedin.com/jobs/view/data-analys...,"Chicago, IL","Job Description\n\n\n\n\nOur client, a leading...",2023-06-01,ENTRY_LEVEL
4,Data Analyst,WellBe Senior Medical,https://www.linkedin.com/jobs/view/data-analys...,"Chicago, IL",The WellBe care model is a Physician Led Advan...,2023-05-01,ENTRY_LEVEL


In [77]:
df['experience'].value_counts()

MID_SENIOR     771
ENTRY_LEVEL    456
ASSOCIATE      150
DIRECTOR        19
EXECUTIVE       18
Name: experience, dtype: int64

In [78]:
# df['description'][97]

In [79]:
s = extract_salary(df)
s[s['salary_str'].map(len)>1]

,id,match,salary_raw,salary_str,salary_numeric
9,21,1,"$75,000 b","[75000, b]",NaN
27,60,3,"$250,000 b","[250000, b]",NaN
28,80,0,$77-100K,"[77, 100000]",88500.0
35,93,0,$38 milli,"[38, milli]",NaN
36,93,1,$4 billi,"[4, billi]",NaN
...,...,...,...,...,...
971,1390,0,$200-600k,"[200, 600000]",400000.0
972,1394,0,$10 billi,"[10, billi]",NaN
977,1396,0,$165 milli,"[165, milli]",NaN
983,1400,1,"$200,000 b","[200000, b]",NaN


In [80]:
s[s['salary_raw'].str.contains('b')]

,id,match,salary_raw,salary_str,salary_numeric
9,21,1,"$75,000 b","[75000, b]",NaN
27,60,3,"$250,000 b","[250000, b]",NaN
36,93,1,$4 billi,"[4, billi]",NaN
59,126,1,$4 billi,"[4, billi]",NaN
64,129,0,$5 billi,"[5, billi]",NaN
...,...,...,...,...,...
953,1369,0,$2B b,"[2b, b]",NaN
955,1369,2,$180k b,"[180000, b]",NaN
970,1386,1,"$250,000 b","[250000, b]",NaN
972,1394,0,$10 billi,"[10, billi]",NaN


## Test extract salary

In [84]:
description = ['$100000', '$100.000', '$100,000', '$100K', '100000K',
               '$100k-200k', '$100-200k', '$100k-$200k', '$100K-200k',
               '$10B', '$10M', '$10 billion', '$15 million', '$100K-a',
               '$100k 200k', '$100 200k', '$100k $200k', '$100K 200k',
               '$100k - 200k', '$100 - 200k', '$100k - $200k', '$100K - 200k',]

df_test = pd.DataFrame(description, columns=['description'])
s = extract_salary(df_test)
s.join(df_test, on='id')

,id,match,salary_raw,salary_str,salary_numeric,description
0,0,0,$100000,[100000],[100000],$100000
1,1,0,$100,[100],[100],$100.000
2,2,0,"$100,000",[100000],[100000],"$100,000"
3,3,0,$100K,[100000],[100000],$100K
4,5,0,$100k-200k,"[100000, 200000]",150000.0,$100k-200k
5,6,0,$100-200k,"[100, 200000]",150000.0,$100-200k
6,7,0,$100k-,[100000],[100000],$100k-$200k
7,7,1,$200k,[200000],[200000],$100k-$200k
8,8,0,$100K-200k,"[100000, 200000]",150000.0,$100K-200k
9,9,0,$10B,[10b],[nan],$10B
